In [1]:
import nltk
from nltk.stem import  WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [3]:
words = []
documents = []
classes = []
ignore_words = ['#', '%', '&', "'", '?', '!', '*', '+', ',', '-', '.', ':', ';', '<', '>']

# Load json file
data_file = open('casino_annotated.json').read()
dialogues = json.loads(data_file)

In [4]:
# iterate through the patterns and tokenize the sentence using nltk.word_tokenize()
nltk.download('punkt')

utterances = {}

for dialogue in dialogues:
    index = 0
    for annotation in dialogue['annotations']:
        tags = annotation[1].split(',')
        for tag in tags:
            if tag != '':
                if not tag in utterances:
                    utterances[tag] = {}
                    utterances[tag]['tag'] = tag
                    utterances[tag]['patterns'] = []
                    utterances[tag]['responses'] = []
                utterances[tag]['patterns'].append(annotation[0])
                if index < len(dialogue['annotations']) - 1:
                    # next utterance
                    response = dialogue['annotations'][index + 1][0]
                else:
                    response = ''
                utterances[tag]['responses'].append(response)
        index = index + 1

for tag in utterances:
    for pattern in utterances[tag]['patterns']:
        # tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)

        # add documents in the corpus
        documents.append((w, utterances[tag]['tag']))   

classes = list(utterances.keys())
print(utterances)

[nltk_data] Downloading package punkt to
[nltk_data]     D:\Users\begumc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'Small-talk': {'tag': 'Small-talk', 'patterns': ['Hello!  It will be so nice to go camping and spend some time in nature during this stressful time.', 'Yes it is. The scenery is beautiful and it is taking my mind off things. So glad to be here. ', 'I am working hard around the camp and getting hungry!', 'You came here to work? I’ve been taking it easy. ', "LOL, I mean setting up the tent and clearing the area for a campfire. I'll take a break now.", 'Sounds good.  I suggest you make a fire and relax. ', 'Hello!', 'Hey there. How are you doing?', 'Great! What do you look for most when camping?', 'Hello, how are you? 🙂', "I'm good, how are you doing today?", "I'm doing great!  I'm on the first camping trip with my five children.", "Wow, that's amazing. My parents took me camping when I was young. Now we going hiking every other weekend. What do you plan on doing?", "That sounds like so much fun!  I hope we get to hike some.  I have my four month old baby with us which is exciting.  I'm 

In [5]:
nltk.download('wordnet')

# lemmatize each word and remove duplicate words from the list
# TODO: remove special characters
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

# sort words
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and utterances
print(len(documents), 'documents')

# classes = tags
print(len(classes), 'tags', classes)

# words = all words, vocabulary
print(len(words), 'unique lemmatized words', words)

# creating a pickle file to store the Python objects which we will use while predicting.
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

[nltk_data] Downloading package wordnet to
[nltk_data]     D:\Users\begumc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


5858 documents
10 tags ['Elicit-Pref', 'No-Need', 'Non-strategic', 'Other-Need', 'Promote-coordination', 'Self-Need', 'Showing-Empathy', 'Small-talk', 'UV-Part', 'Vouch-Fair']
2936 unique lemmatized words ["''", "'d", "'food", "'good'.🙂", "'items", "'ll", "'m", "'re", "'s", "'ve", '(', ')', ',2', '--', '-10', '..', '...', '....', '.....', '......', '.......', '.hope', '.thats', '.there', '.🙂', '/', '0', '1', '1-1', '1-2', '1.is', '1/2', '1/3', '10', '100', '11', '110-118', '115', '12', '13', '14', '15', '17', '1water', '1wood', '2', '2-1', '2/1', '2/3', '21', '260-pound', '2:1', '2food', '2water', '3', '3/0', '3x', '4', '40', '5', '50', '5000', '6', '6-foot-4', '6/9', '7', '70', '75', '8', '``', 'a', 'ability', 'able', 'abou', 'about', 'above', 'absolute', 'absolutely', 'absolutly', 'ac', 'acadia', 'accept', 'acceptable', 'accepted', 'accepting', 'access', 'accessible', 'accidentally', 'accommodate', 'accommodating', 'accomodate', 'accomplish', 'achieve', 'acquiring', 'across', 'active

In [6]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0) 
    
    # output is '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[: , 0])
train_y = list(training[:, 1])

print("Training data created")
print("training data shape", training.shape)

Training data created
training data shape (5858, 2)


d:\users\begumc\data-mining\lib\site-packages\ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [7]:
# create 3 layers. First layer 128 neurons,
# second layer 64 neurons
# 3rd output layer contains number of neurons equal to number of tags to predict output tag with softmax


model = Sequential()
model.add(Dense(128, 
                input_shape=(len(train_x[0]),),
                activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


# Compile model. Stochastic gradient descent with Nesterov accelerated gradient 
sgd = SGD(learning_rate = 0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer = sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), 
                 np.array(train_y), 
                 epochs=200, 
                 batch_size=5, 
                 verbose=1)
model.save('model.h5', hist)
print('Model created')

Epoch 1/200
1172/1172 [==============================] - 30s 5ms/step - loss: 1.9334 - accuracy: 0.3623
Epoch 2/200
1172/1172 [==============================] - 5s 4ms/step - loss: 1.6312 - accuracy: 0.4647
Epoch 3/200
1172/1172 [==============================] - 5s 5ms/step - loss: 1.5565 - accuracy: 0.4809
Epoch 4/200
1172/1172 [==============================] - 5s 4ms/step - loss: 1.5064 - accuracy: 0.4948
Epoch 5/200
1172/1172 [==============================] - 5s 4ms/step - loss: 1.4821 - accuracy: 0.5011
Epoch 6/200
1172/1172 [==============================] - 5s 4ms/step - loss: 1.4630 - accuracy: 0.5117
Epoch 7/200
1172/1172 [==============================] - 5s 4ms/step - loss: 1.4048 - accuracy: 0.5353
Epoch 8/200
1172/1172 [==============================] - 5s 4ms/step - loss: 1.4009 - accuracy: 0.5342
Epoch 9/200
1172/1172 [==============================] - 5s 4ms/step - loss: 1.3609 - accuracy: 0.5500
Epoch 10/200
1172/1172 [==============================] - 5s 4ms/step - 

1172/1172 [==============================] - 5s 4ms/step - loss: 0.9973 - accuracy: 0.6411
Epoch 80/200
1172/1172 [==============================] - 5s 4ms/step - loss: 0.9827 - accuracy: 0.6371
Epoch 81/200
1172/1172 [==============================] - 5s 4ms/step - loss: 0.9955 - accuracy: 0.6403
Epoch 82/200
1172/1172 [==============================] - 5s 4ms/step - loss: 0.9899 - accuracy: 0.6447
Epoch 83/200
1172/1172 [==============================] - 5s 4ms/step - loss: 0.9769 - accuracy: 0.6428
Epoch 84/200
1172/1172 [==============================] - 5s 4ms/step - loss: 0.9917 - accuracy: 0.6394
Epoch 85/200
1172/1172 [==============================] - 5s 4ms/step - loss: 0.9887 - accuracy: 0.6388
Epoch 86/200
1172/1172 [==============================] - 5s 4ms/step - loss: 0.9701 - accuracy: 0.6514
Epoch 87/200
1172/1172 [==============================] - 5s 4ms/step - loss: 0.9474 - accuracy: 0.6498
Epoch 88/200
1172/1172 [==============================] - 5s 4ms/step - loss:

1172/1172 [==============================] - 5s 4ms/step - loss: 0.8592 - accuracy: 0.6734
Epoch 157/200
1172/1172 [==============================] - 5s 4ms/step - loss: 0.8865 - accuracy: 0.6628
Epoch 158/200
1172/1172 [==============================] - 5s 4ms/step - loss: 0.8715 - accuracy: 0.6660
Epoch 159/200
1172/1172 [==============================] - 5s 4ms/step - loss: 0.8880 - accuracy: 0.6736
Epoch 160/200
1172/1172 [==============================] - 4s 4ms/step - loss: 0.8809 - accuracy: 0.6632: 0s - loss:
Epoch 161/200
1172/1172 [==============================] - 5s 4ms/step - loss: 0.8900 - accuracy: 0.6600
Epoch 162/200
1172/1172 [==============================] - 5s 4ms/step - loss: 0.8513 - accuracy: 0.6705
Epoch 163/200
1172/1172 [==============================] - 5s 4ms/step - loss: 0.8930 - accuracy: 0.6614
Epoch 164/200
1172/1172 [==============================] - 5s 4ms/step - loss: 0.8451 - accuracy: 0.6873
Epoch 165/200
1172/1172 [==============================] 

In [8]:
from keras.models import load_model
model = load_model('model.h5')
import random

annotations = []
for tag in utterances:
    annotations.append(utterances[tag])

words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, annotations):
    tag = ints[0]['intent']
    list_of_intents = annotations
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, annotations)
    return res



In [17]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        print(res)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("Negotiate")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview)
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

I can use water, yes. You could have all the firewood, one food ration. Then I could take the water rations and two food rations. 
